# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

Identifying students who might need early intervention before they fail to graduate would be an example of classification because we're looking for a discrete outcome of only two possibilities: passed, failed.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [28]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
# For slkearn >= 0.18 use the following instead:
#from sklearn.model_selection import ShuffleSplit

# Read student data
student_data = pd.read_csv("student-data.csv")

# Student outcomes
y = student_data['passed']

print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [29]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data["passed"] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data["passed"] == "no"])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [30]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [31]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        # If data type is categorical, convert to dummy variables; example: 'school' => 'school_GP' and 'school_MS'
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
            col_data = pd.get_dummies(col_data, prefix = col)
              
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (56 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup_0', 'schoolsup_1', 'famsup_0', 'famsup_1', 'paid_0', 'paid_1', 'activities_0', 'activities_1', 'nursery_0', 'nursery_1', 'higher_0', 'higher_1', 'internet_0', 'internet_1', 'romantic_0', 'romantic_1', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [32]:
# TODO: Set the number of training points
num_train = 300

print X_all.shape

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "Graduation rate of the training set: {:.2f}%".format(100 * (y_train == "yes").mean())
print "Graduation rate of the testing set: {:.2f}%".format(100 * (y_test == "yes").mean())

(395, 56)
Training set has 300 samples.
Testing set has 95 samples.
Graduation rate of the training set: 68.67%
Graduation rate of the testing set: 62.11%


## Training and Evaluating Models
In this section, you will choose at least 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List at least three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

I'm listing other commonly used Supervised Learning algorithms in the cell below, along with their strengths and weaknesses for future reference but the three scikit-learn models I would choose for this project are:

Gaussian Naive Bayes (GaussianNB):
- Naive Bayes Classifiers are commonly used in text classification, word processing, business analysis and marketing. A well known use would be a spam filter for email.
- Even faster than linear models, good for very large datasets and high-dimensional data.
- Often less accurate than linear models.
- Good for this project because Naive Bayes Classifiers do well with data in which the inputs are independent from one another. 

Decision Trees:
- Decision Trees have been used in computer vision, robotics, control systems, video games and medical analysis. Binary search trees (B-trees), a special type of binary tree, are commonly used to store data in databases.
- Decision Trees are very fast, don’t need scaling of the data; can be visualized and easily explained.
- Downsides are that you don't get random access and performance times are on average and rely on the tree being balanced.
- DecisionTreeClassifier could be a good option here because the dataset is small and by using Stratified Shuffle Split (SSS) we can correct the imbalance of the training data (265 passed; 130 failed).

K-Nearest Neighbors (KNeighbors):
- kNN has been used for OCR (optical character recognition) by extracting the features of characters/numbers and matching them up with their nearest known (labeled) neighbor. A common use is for recommending products to consumers based on preferences of customers similar to them.
- For small datasets, kNN is good as a baseline and easy to explain.
- Weaknesses: Need to give a good similarity measure which may not be known apriori; query/prediction cost is high depending on the value of k chosen; curse of dimensionality.
- Good for this project to set up a quick and simple algorithm since the training set is quite small, so we can try out kNN for fast prediction times.

Logistic Regression:
- The Injury Severity Score (ISS) would be a good example of a real world application of logistic regression (which is what was used when it was originally developed. It correlates with mortality, morbidity and hospitalization time after trauma.
- Go-to as a first algorithm to try, good for very large datasets; good for very high-dimensional data.
- Linear model and cannot capture any complex relationships between the data and the target variable. It is a high bias model and given sufficient data, other more complex models with less bias may outperform it. Logistic regression cannot predict continuous outcomes.
- Logistic Regression is easy to apply and a solid baseline to compare other more complex methods. Thus, it should be the first thing to start with when looking for an optimal classifier.


REFERENCES:

- An Introduction to Machine Learning with Python by Andreas C. Müller and Sarah Guido (O’Reilly). Copyright 2017 Sarah Guido and Andreas Müller, 978-1-449-36941-5

- Toughtful Machine Learning by Matthew Kirk (O’Reilly). Copyright 2015 Matthew Kirk, 978-1-449-37406-8

Additional Models:

-Random Forests:
Nearly always perform better than a single decision tree, very robust and powerful. Don’t need scaling of data. Not good for very high-dimensional, sparse data.

-Gradient Boosted Decision Trees:
Often slightly more accurate than random forests. Slower to train but faster to predict than random forests, and smaller in memory. Need more parameter tuning than random forests.

-Support Vector Machines:
Powerful for medium-sized datasets of features with similar meaning. Require scaling of data; sensitive to parameters.

Additionally, SVMs come with two theoretical benefits: 1) A guaranteed optimal solution (you can get this with simpler techniques like logistic regression). 2) The ability to use non-linear kernels, which can offer much more power than logistic regression (on par with neural networks).

In practice though, non-linear kernels slow training down to the point of being impractical in many cases. Linear kernels may be fast enough, but remove the second benefit (mentioned above), so most people would prefer to use established linear techniques like logistic regression.

-Neural Networks:
Can build very complex models, particularly for large datasets. Sensitive to scaling of the data and to the choice of parameters. Large models need a long time to train.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [33]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

def stratify_shuffle(X, y, num_train):
    sss = StratifiedShuffleSplit(y, 1, train_size=num_train, random_state=0)

    # just one iteration
    for train_index, test_index in sss:
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index] 
        
    return X_train, X_test, y_train, y_test

print("Done")

Done


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [34]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state = 0)
clf_C = KNeighborsClassifier(n_neighbors=5)
clf_D = svm.SVC(random_state=0)
clf_E = LogisticRegression(random_state=0)
classifiers = [clf_A, clf_B, clf_C, clf_D, clf_E]

# TODO: Set up the training set sizes
# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train # entire training set
y_train_300 = y_train

#X_train_100, X_test_100, y_train_100, y_test_100 = stratify_shuffle(X_train, y_train, 100)
#X_train_200, X_test_200, y_train_200, y_test_200 = stratify_shuffle(X_train, y_train, 200)
#X_train_300, X_test_300, y_train_300, y_test_300 = X_train, X_test, y_train, y_test

training_sets = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

for clf in classifiers:
    for X, y in training_sets:
        train_predict(clf, X, y, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0034 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.8682.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7576.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0028 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 0.8000.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.7176.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0026 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8058.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7391.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0023 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7200.
Training a DecisionTreeClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0034                       | 0.0012                       | 0.8682                 | 0.7576                |
| 200               | 0.0028          | 0.0011                       | 0.8000                 | 0.7176                |
| 300               | 0.0026                        | 0.0005                       | 0.8058                 | 0.7391      

** Classifer 2 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0023                        | 0.0006                       | 1.000                 | 0.7200                |
| 200               | 0.0021             | 0.0004                       | 1.000                 | 0.7231                |
| 300               | 0.0069                        | 0.0006                       | 1.000                 | 0.7333     |

** Classifer 3 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0027                        | 0.0018                       | 0.8000                 | 0.7176                |
| 200               | 0.0015                        | 0.0030                       | 0.8522                 | 0.7015                |
| 300               | 0.0017                        | 0.0031                       | 0.8767                 | 0.7518         |

** Classifer 4 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0019                        | 0.0009                       | 0.8533                 | 0.7785                |
| 200               | 0.0041          | 0.0016                       | 0.8636                 | 0.7755                |
| 300               | 0.0086                        | 0.0018                       | 0.8674                 | 0.7586      |


** Classifer 5 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0019                        | 0.0005                       | 0.8696                 | 0.7612                |
| 200               | 0.0023          | 0.0004                       | 0.8410                 | 0.7941                |
| 300               | 0.0027                        | 0.0004                       | 0.8337                 | 0.7761      |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I would choose Logistic Regression (LR) for the classifier. It has the highest test F1 score when using a training set size of 200, as well as when using the full training set. SVC has somewhat competitive test F1 scores but it takes significantly longer to train and make predictions. LR is faster and it's a less complex model, suitable for the data in hand (majority of the data is linearly separable). The table above confirms that it is the fastest of all the models in terms of prediction time and its superior accuracy over kNN outweighs its longer training time. Also, LR's spread between F1 scores on training and test sets are the lowest. This is an indicator that LR is not overfitting to the small training set as much as some of the other models.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

The Logistic Regression model is able to predict how likely a student is to graduate. It does so by analyzing the 395-student data (that have already been through the program). By looking at the attributes of these students, our model can identify cues that have been idicative of failing to graduate in the past. We can look for these cues in future students to get our intervention system.

To get a feeling for how the model works, consider the concept of an extraordinarily high number of absences correlating closely with a high failure rate. This is quite intuitive and a rather obvious relationship. What our model does is expand on this idea and look for more subtle, less obvious relationships and patterns, using many more variables than just absences. Because we have detailed records on previous students and we know whether each of those students passed or failed, we can use this information to train our model on what a passing student and what a failing student "looks like" so to speak. The model will identify predictive factors and assign weights (higher for those with more influence; less for those with less influence on the outcome) to each and evaluate their influence on the outcome of each of the former students. That's the formal training process. Once that's done we can feed it records of existing students and our model can develop a probability of failure for each current student. If the probability of failure is above a certain threshold the model will identify that student as one in need of additional guidance. Put simply, we will train our model to recognize and identify students who appear to be similar to past students that have failed to graduate.

Over time, and with more student records entered, this model will continue to "learn" and get better and better at predicting which students are at risk of not graduating.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [37]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = [{'C': np.logspace(0.00001, 5, num=100), 'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None]}]

# TODO: Initialize the classifier
clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, cv=5, verbose=0)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0007 seconds.
Tuned model has a training F1 score of 0.8363.
Made predictions in 0.0004 seconds.
Tuned model has a testing F1 score of 0.7737.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The tuned model has a training F1 score of 0.8363 and a testing F1 score of 0.7737. This is nearly identical to the untuned F1 scores of 0.8337 and 0.7761 for training and testing, respectively. I suspect with a larger dataset we'd see more of an impact from fine tuning out parameters.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.